In [75]:
import os
import urllib
import pyodbc
import sqlalchemy
import pandas as pd
import numpy as np
import pyreadr
import geopandas as gpd

In [163]:
# output directory
output_dir = r'C:\Workspace\VisionEval\psrc_scripts\survey_data'


In [347]:
mode_dict = {
    1: 1,    # 01	Walk	81,288	8.8
    2: 2,    # 02	Bicycle	8,034	0.9
    3: 3,    # 03	Car	396,931	43.0
    4: 3,    # 04	SUV	229,466	24.8
    5: 3,    # 05	Van	60,463	6.5
    6: 11,    # 06	Pickup truck	108,303	11.7
    8: 10,    # 07	Golf cart / Segway	826	0.1
    9: 97    # 08	Motorcycle / Moped	2,088	0.2
        # 09	RV (motor home, ATV, snowmobile)	814	0.1
        # 10	School bus	11,313	1.2
        # 11	Public or commuter bus	6,616	0.7
        # 12	Paratransit / Dial-a-ride	624	0.1
        # 13	Private / Charter / Tour / Shuttle bus	1,581	0.2
        # 14	City-to-city bus (Greyhound, Megabus)	120	0.0
        # 15	Amtrak / Commuter rail	1,148	0.1
        # 16	Subway / elevated / light rail / street car	3,326	0.4
        # 17	Taxi / limo (including Uber / Lyft)	2,813	0.3
        # 18	Rental car (Including Zipcar / Car2Go)	2,006	0.2
        # 19	Airplane	1,823	0.2
        # 20	Boat / ferry / water taxi	458	0.0
        # 97	Somethi
}

purp_dict = {
    0: 1,    # 01	Regular home activities (chores, sleep)	307,252	33.3
    1: 3,    # 02	Work from home (paid)	8,822	1.0
    2: 8,    # 03	Work	100,012	10.8
    3: 6,    # 04	Work-related meeting / trip	11,812	1.3
    4: 14,    # 05	Volunteer activities (not paid)	8,533	0.9
    5: 12,    # 06	Drop off /pick up someone	56,615	6.1
    6: 13,    # 07	Change type of transportation	9,536	1.0
    7: 17,    # 08	Attend school as a student	22,645	2.5
    10: 7    # 09	Attend child care	2,082	0.2
        # 10	Attend adult care	560	0.1
        # 11	Buy goods (groceries, clothes, appliances, gas)	132,851	14.4
        # 12	Buy services (dry cleaners, banking, service a car, pet care)	23,592	2.6
        # 13	Buy meals (go out for a meal, snack, carry-out)	72,391	7.8
        # 14	Other general errands (post office, library)	28,126	3.0
        # 15	Recreational activities (visit parks, movies, bars, museums)	31,394	3.4
        # 16	Exercise (go for a jog, walk, walk the dog, go to the gym)	31,518	3.4
        # 17	Visit friends or relatives	36,869	4.0
        # 18	Health care visit (medical, dental, therapy)	16,967	1.8
        # 19	Religious or other community activities	18,896	2.0
        # 97	Something else
}

hometype_map = {
        # -7=Refused 16 11
        # -8=Don't Know 18 5
        1: 1,# 1=Detached single house 122
        2: 2,# 2=Duplex 6
        3: 4,# 3=Rowhouse or townhouse 6
        4: 5,# 4=Apartment  condominium
        5: 6,# 5=Mobile home or trailer 6
        6: 91,# 6=Dorm room  fraternity or sorority house
        9: 91# 91=Other
}


# Household

In [681]:
# Daysim Survey location
daysim_loc = r'R:\e2projects_two\SoundCast\Inputs\dev\base_year\2018\survey'
daysim_hh = pd.read_csv(os.path.join(daysim_loc,'_household.tsv'), delim_whitespace=True)
daysim_person = pd.read_csv(os.path.join(daysim_loc,'_person.tsv'), delim_whitespace=True)

In [682]:
# Load NHTS survey files
fname = r'C:\Workspace\VisionEval_build\VisionEval\sources\modules\VE2001NHTS\data-raw\Hh_df.Rda'
nhts_hh = pyreadr.read_r(fname)['Hh_df']

In [683]:

ve_hh = pd.pivot_table(daysim_person, index='hhno', columns='pno', values='pagey', aggfunc='mean').reset_index()
for i in range(1,15):
    if i not in ve_hh.columns:
        ve_hh['AGE_P'+str(i)] = -1
    ve_hh.rename(columns={i: 'AGE_P'+str(i)}, inplace=True)
ve_hh.rename(columns={'hhno': 'HOUSEID'}, inplace=True)

ve_hh = ve_hh.merge(daysim_person, left_on='HOUSEID', right_on='hhno', how='left')
ve_hh = ve_hh.merge(daysim_hh,  on='hhno', how='left')

In [684]:
# 'CENSUS_D':  household Census division
# division 9; https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf
ve_hh['CENSUS_D'] = 9

# 'CENSUS_R': houeshold census region
# region 4;
ve_hh['CENSUS_R'] = 4

# 'EXPFLHHN', HH Weight-100 percent completed - NATL
# use hhexpfac?
ve_hh['EXPFLHHN'] = ve_hh['hhexpfac'].copy()

# 'EXPFLLHH',HH Weight-100 percent completed
ve_hh['EXPFLLHH'] = ve_hh['hhexpfac'].copy()

#  'FLGFINCM', Incomes of all HH members included?
# Assume yes 
ve_hh['FLGFINCM'] = 1

# 'HHC_MSA', MSA / CMSA code for HH
# FIXME: there is an MSA code for bremerton/kitsap county that should be used in some cases
# 42660: https://www.hud.gov/sites/documents/11-39MLATCH2.PDF
ve_hh['HHC_MSA'] = 42660

# 'HHNUMBIK', Number of full size bicycles in HH
# FIXME: get this from the NHTS average for our area or some other source
ve_hh['HHNUMBIK'] = 0

# 'HHR_DRVR',Driver status of HH respondent  | 1 y, 2 n
# FIXME: for now assume all respondents are drivers
ve_hh['HHR_DRVR'] = 1

# 'HHSIZE', Count of HH members
ve_hh['HHSIZE'] = ve_hh['hhsize'].copy()

# 'HHVEHCNT', Count of vehicles in HH
ve_hh['HHVEHCNT'] = ve_hh['hhvehs'].copy()

# 'HOMETYPE', Type of housing unit
ve_hh['HOMETYPE'] = ve_hh['hrestype'].map(hometype_map)

# 'MSAPOP', Number of persons residing in the MSA
# Estimating 3.8 M in 2018?
# FIXME: get the actual number and account for bremerton MSA
ve_hh['MSAPOP'] = 3800000

# MSACAT
# 1=MSA of 1 million or more, with rail
# 2=MSA of 1 million or more, and not in 1
# 3=MSA less than 1 million
# 4=Not in MSA (CMSA)
ve_hh['MSACAT'] = 1

# 'MSASIZE', MSA size; https://nhts.ornl.gov/tables09/CodebookPage.aspx?id=1257
ve_hh['MSASIZE'] = 5

# 'RAIL', Rail (subway) category. whether the household is located in a MSA with rail.
ve_hh['RAIL'] = 1

# 'URBAN', Household in urbanized area | Household's urban area classification, based on home address and 2014 TIGER/Line Shapefile
# 1=In an Urban cluster
# 2=In an urban area
# 3=In an area surrounded by urban areas
# 4=Not in urban area
# FIXME: need to do this for each household?
# Assume all urban for now
ve_hh['URBAN'] = 1

# 'URBRUR', Household in urban/rural area | 1 URBAN , 2 RURAL
# We can get this from ElmerGeo definitions
ve_hh['URBRUR'] = 1




# 'CNTTDHH' 
# Sum of all travel period person trips. sum of all Travel Day person trips enumerated
# (G12 through G14) by interviewed household members in useable households
# Total trips, can be produced by aggregating the trips file
tot_trips = daysim_trip.groupby('hhno').count()[['trexpfac']].reset_index()
ve_hh = ve_hh.merge(tot_trips, on='hhno', how='left')
ve_hh.rename(columns={'trexpfac': 'CNTTDHH'}, inplace=True)

In [685]:
# 'HHFAMINC', Total HH income last 12 months (category)
ve_hh['HHFAMINC'] =pd.cut(ve_hh['hhincome'], bins=[-1,10000,15000,25000,35000,49000,75000,100000,125000,150000,200000,99999999999],
                        labels=[1,2,3,4,5,6,7,8,9,10,11])

# # 'HHINCTTL', Total income all HH members (category)
ve_hh['HHINCTTL'] = ve_hh['HHFAMINC'].copy()




In [686]:


# 'WRKCOUNT',Count of HH members with jobs
ve_hh['WRKCOUNT'] = ve_hh['hhwkrs']


# WKR_P1 - WKR_P14
daysim_person['is_worker'] = 2
daysim_person.loc[daysim_person['pwtyp'] > 0, 'is_worker'] = 1

df = pd.pivot_table(daysim_person, index='hhno', columns='pno', values='is_worker', aggfunc='sum').reset_index()
for i in range(1,15):
    if i not in df.columns:
        df['WKR_P'+str(i)] = -1
    df.rename(columns={i: 'WKR_P'+str(i)}, inplace=True)

ve_hh = ve_hh.merge(df, on='hhno', how='left')

# 'DRVRCNT': drivers in household
# FIXME: Get from Elmer
# for now assume all people >= 16 are drivers
df = daysim_person[daysim_person['pagey'] >= 16]
df = df.groupby('hhno').count()[['psexpfac']].reset_index()
df.rename(columns={'psexpfac': 'DRVRCNT'}, inplace=True)
ve_hh = ve_hh.merge(df, on='hhno', how='left')

# 'DRV_P1' - 'DRV_P14', driver status
# 1 y, 2 n
# Again, assuming all 16+ people are drivers
daysim_person['is_driver'] = 2
daysim_person.loc[daysim_person['pagey'] >= 16, 'is_driver'] = 1
df = pd.pivot_table(daysim_person, index='hhno', columns='pno', values='is_driver', aggfunc='sum').reset_index()
for i in range(1,15):
    if i not in df.columns:
        df['DRV_P'+str(i)] = -1
    df.rename(columns={i: 'DRV_P'+str(i)}, inplace=True)
    # ve_hh['DRV_P'+str(i)].replace(0,2, inplace=True)
ve_hh = ve_hh.merge(df, on='hhno', how='left')

# 'RATIO16V', Ratio - HH members (16+) to vehicles
ve_hh['RATIO16V'] = ve_hh['DRVRCNT']/ve_hh['hhvehs']
ve_hh['RATIO16V'] = ve_hh['RATIO16V'].replace(np.inf, 0)

In [687]:
# 'HHR_AGE', Respondent age
# Use oldest member of the household
df = daysim_person.groupby('hhno').max()[['pagey']].reset_index()
df.rename(columns={'pagey': 'HHR_AGE'}, inplace=True)
ve_hh = ve_hh.merge(df, on='hhno', how='left')

# 'HHR_RACE', Race of HH respondent
# FIXME: 
ve_hh['HHR_RACE'] = 1

# 'HHR_SEX', Gender of HH respondent
# FIXME: change this if it matters for estimation?
ve_hh['HHR_SEX'] = 1


In [688]:
# 'LIF_CYC', HH Life Cycle
# FIXME: calculate this for the household
ve_hh['LIF_CYC'] =-9

# -9=Not Ascertained	
# 01=one adult, no children	
# 02=2+ adults, no children	
# 03=one adult, youngest child 0-5	
# 04=2+ adults, youngest child 0-5	
# 05=one adult, youngest child 6-15	
# 06=2+ adults, youngest child 6-15	
# 07=one adult, youngest child 16-21	
# 08=2+ adults, youngest child 16-21	
# 09=one adult, retired, no children	
# 10=2+ adults, retired, no children	

In [689]:
# ve_hh['RATIO16V'].unique()

In [690]:
df_lu = pd.read_csv(r'R:\e2projects_two\SoundCast\Inputs\dev\landuse\2018\v3_RTP\parcels_urbansim.txt', delim_whitespace=True)
df_lu.columns

Index(['aparks', 'empedu_p', 'empfoo_p', 'empgov_p', 'empind_p', 'empmed_p',
       'empofc_p', 'empoth_p', 'empret_p', 'emprsc_p', 'empsvc_p', 'emptot_p',
       'hh_p', 'lutype_p', 'mfunits', 'nparks', 'parcelid', 'parkdy_p',
       'parkhr_p', 'ppricdyp', 'pprichrp', 'sfunits', 'sqft_p', 'stugrd_p',
       'stuhgh_p', 'stuuni_p', 'taz_p', 'xcoord_p', 'ycoord_p'],
      dtype='object')

In [691]:
# For each TAZ? 
df = df_lu.groupby('taz_p').sum()[['hh_p']]
df2 = df_lu.groupby('taz_p').sum()[['sqft_p']]
df2['sq_mile'] = df2['sqft_p']/(5280**2)
df = df.merge(df2,  left_index=True, right_index=True)
df['HBHRESDN'] = df['hh_p']/df['sq_mile']
df['HBHRESDN'].max()

# Load parcel geography lookup table
parcel_lookup = pd.read_csv(r'R:\e2projects_two\SoundCast\Inputs\db_inputs\parcel_2018_geography.csv')

In [692]:
df_lu = df_lu.merge(parcel_lookup, left_on='parcelid', right_on='ParcelID')

In [693]:
# df_lu.columns

In [694]:

# The following data must be provided by land use data
# 'HBHRESDN', Housing units per sq mile - Block group. Housing units density in units per square mile provided by Claritas

# 'HBHUR', Urban / Rural indicator - Block group
# Fixme: get this definition from land use
ve_hh['HBHUR'] = 'U'

# 'HBPPOPDN', Population per sq mile - Block group

# 'HTEEMPDN',Workers per square mile living in Tract

# 'HTHRESDN', Housing units per sq mile - Tract level
# 'HTHUR', Urban / Rural indicator - Tract level
ve_hh['HTHUR'] = 'U'
# 'HTPPOPDN', Population per sq mile - Tract level

In [695]:
# ve_hh.columns

In [696]:
# For now just set these all at 1
ve_hh[['HBHRESDN', 'HBPPOPDN', 'HTEEMPDN', 'HTHRESDN','HTPPOPDN']] = 3000



In [699]:
# Floats
# 
float_cols = ['EXPFLHHN','EXPFLLHH','HBHUR','HTHUR','HHFAMINC','HHINCTTL','CENSUS_D','CENSUS_R']
ve_hh[nhts_hh.columns.drop(float_cols)] = ve_hh[nhts_hh.columns.drop(float_cols)].fillna(-1).astype('int')
df = ve_hh[nhts_hh.columns]

df['HOUSEID'] = df['HOUSEID'].astype('str')
pyreadr.write_rdata(output_dir + '\\Hh_df.Rda', df, df_name="Hh_df")

c:\Users\Modeller\.conda\envs\summary\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
ve_hh['HBHUR']


0        1
1        1
2        1
3        1
4        1
        ..
11925    1
11926    1
11927    1
11928    1
11929    1
Name: HBHUR, Length: 11930, dtype: int64

# Persons

In [591]:
# Load NHTS survey files
fname = r'C:\Workspace\VisionEval_build\VisionEval\sources\modules\VE2001NHTS\data-raw\Per_df.Rda'
nhts_person = pyreadr.read_r(fname)['Per_df']

In [592]:
# HOUSEID	
daysim_person['HOUSEID'] = daysim_person['hhno'].astype('str')
# PERSONID	
daysim_person['PERSONID'] = daysim_person['pno'].astype('str')

# COMMDRVR	Person is a commercial driver
# FIXME: get this from Elmer
# Assuming no for all persons for now
daysim_person['COMMDRVR'] = 2

# NBIKETRP	Number of bike trips in past week
# FIXME: get this from Elmer
# For now extrapolate based on daysim data; if they rode a bike on the current day, assume 3 otherwise 0
daysim_trip['PERSONID']  = daysim_trip['hhno'].astype('str') + daysim_trip['pno'].astype('str')
bike_trips = daysim_trip[daysim_trip['mode'] == 2]
bike_trips = bike_trips.groupby('PERSONID').count()[['trexpfac']].reset_index()
bike_trips.rename(columns={'trexpfac': 'NBIKETRP'}, inplace=True)
daysim_person = daysim_person.merge(bike_trips, how='left', on='PERSONID')
daysim_person['NBIKETRP'].fillna(0, inplace=True)
daysim_person['NBIKETRP']= daysim_person['NBIKETRP']*4     # Average for the week

# NWALKTRP	Number of walk trips in past week
# FIXME: get this from Elmer
walk_trips = daysim_trip[daysim_trip['mode'] == 1]
walk_trips = walk_trips.groupby('PERSONID').count()[['trexpfac']].reset_index()
walk_trips.rename(columns={'trexpfac': 'NWALKTRP'}, inplace=True)
daysim_person = daysim_person.merge(walk_trips, how='left', on='PERSONID')
daysim_person['NWALKTRP'].fillna(0, inplace=True)
daysim_person['NWALKTRP']= daysim_person['NWALKTRP']*4    # Average for the week

# WRKDRIVE	Job requires driving a motor vehicle
# FIXME: get this from Elmer
# Assuming no for all persons for now
daysim_person['WRKDRIVE'] = 2

# WORKER	Person has job (1=yes, 0=no)
daysim_person['WORKER'] = 0
daysim_person.loc[daysim_person['pwtyp'] > 0, 'WORKER'] = 1

# DTGAS	
# Not sure of the source of this field; in original data it's an integer
# potentially PRICE field in NHTS codebook; influence of gas price on choices
daysim_person['DTGAS'] = 3

# DISTTOWK	Distance to work in miles
daysim_person['DISTTOWK'] = daysim_person['pwaudist'].copy()

# DRIVER	Driver status (1=driver, 0=non-driver)
daysim_person['DRIVER'] = 2
daysim_person.loc[daysim_person['pagey'] >= 16, 'DRIVER'] = 1

# R_AGE	Respondent age
daysim_person['R_AGE'] = daysim_person['pagey'].copy()

# R_SEX	R_sex
# Male 1; female 2
daysim_person['R_SEX'] = daysim_person['pgend'].copy()


In [593]:

# USEPUBTR	Used public transit on travel day
# FIXME: lookup from trip file
daysim_trip['PERSONID'] = daysim_trip['hhno'].astype('str')+daysim_trip['pno'].astype('str')
df = daysim_trip[daysim_trip['mode'] == 6]
daysim_person['USEPUBTR'] = 2
daysim_person.loc[daysim_person['PERSONID'].isin(df['PERSONID'].unique()),'USEPUBTR'] = 1

# WRKTRANS	Transportation mode to work last week
# FIXME: Get usual commute mode from Elmer
df = daysim_trip[daysim_trip['dpurp'] == 1]
df = df.groupby('PERSONID').first()[['mode']].reset_index()
df.rename(columns={'mode': 'WRKTRANS'}, inplace=True)
daysim_person = daysim_person.merge(df, on='PERSONID', how='left')
daysim_person['WRKTRANS'] = daysim_person['WRKTRANS'].map(mode_dict)
daysim_person['WRKTRANS'].fillna(-1, inplace=True)


In [594]:
daysim_person[nhts_person.columns.drop('DISTTOWK')] = daysim_person[nhts_person.columns.drop('DISTTOWK')].astype('int')
df = daysim_person[nhts_person.columns]
df['HOUSEID'] = df['HOUSEID'].astype('str')
pyreadr.write_rdata(output_dir + '\\Per_df.Rda', df, df_name="Per_df")

c:\Users\Modeller\.conda\envs\summary\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [431]:
df

,HOUSEID,PERSONID,COMMDRVR,NBIKETRP,NWALKTRP,USEPUBTR,WRKDRIVE,WRKTRANS,WORKER,DTGAS,DISTTOWK,DRIVER,R_AGE,R_SEX
0,17100005,171000051,2,0,0,2,2,-1,0,3,-1.00,1,60,2
1,17100005,171000052,2,0,0,2,2,-1,0,3,-1.00,1,70,1
2,17100024,171000241,2,8,16,2,2,1,1,3,0.14,1,30,9
3,17100024,171000242,2,0,4,1,2,1,1,3,1.37,1,30,2
4,17100024,171000243,2,4,0,2,2,-1,0,3,-1.00,2,2,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11925,192018397,1920183971,2,0,8,2,2,-1,1,3,-1.00,1,30,1
11926,192018397,1920183972,2,0,0,2,2,3,1,3,4.24,1,30,2
11927,192018425,1920184252,2,0,8,2,2,3,1,3,11.38,1,30,1
11928,192018425,1920184251,2,0,0,2,2,-1,1,3,-1.00,1,30,2


# Tour

In [700]:

daysim_tour = pd.read_csv(os.path.join(daysim_loc,'_tour.tsv'), delim_whitespace=True)
daysim_person = pd.read_csv(os.path.join(daysim_loc,'_person.tsv'), delim_whitespace=True)

# Load NHTS survey files
fname = r'C:\Workspace\VisionEval_build\VisionEval\sources\modules\VE2001NHTS\data-raw\ToursByHh_df.Rda'
nhts_tour = pyreadr.read_r(fname)['ToursByHh_df']

In [701]:
# daysim_tour[['tripsh1','tripsh2']].sum(axis=1)

<!-- Houseid	Unique household ID
Distance	Total distance in miles of the tour
TravelTime	Total time in minutes spent traveling on the tour
DwellTime	Total time in minutes spent at activities on the tour
StartHome	Logical identifying if the tour started at home
EndHome	Logical identifying if the tour ended at home
Trips	Number of trips in the tour
Persons	Number of persons on the tour
Vehid	Unique ID for vehicle in household
Trptrans	Mode of transportation (see 2001 NHTS codebook for TRPTRANS)
Vehtype	Type of vehicle used (see 2001 NHTS codebook for VEHTYPE)
HhVehUsed	Whether household vehicle used (1=yes, 2=no)
Whyto	String contenating successive activity codes at trip end (see 2001 NHTS codebook for WHYTO)
Disttowk	Distance from home to work for the person on the tour who works farthest from home -->


In [702]:
# Houseid	Unique household ID
daysim_tour['Houseid'] = daysim_tour['hhno'].astype('str')

# Distance	Total distance in miles of the tour
daysim_tour['Distance'] = (daysim_tour['d']/100.0).copy()

# TravelTime	Total time in minutes spent traveling on the tour
daysim_tour['TravelTime'] = (daysim_tour['t']/100.0).copy()

# DwellTime	Total time in minutes spent at activities on the tour
daysim_tour['DwellTime'] = daysim_tour['tarorig'] - daysim_tour['tlvorig']

# StartHome
daysim_tour[['StartHome','EndHome']] = False
daysim_tour.loc[daysim_tour['pdpurp'] != 0, 'StartHome'] = True

# EndHome
daysim_tour.loc[daysim_tour['pdpurp'] == 0, 'EndHome'] = True

# Trips
daysim_tour['Trips'] = daysim_tour[['tripsh1','tripsh2']].sum(axis=1)

# Vehid
# FIXME use Elmer to get vehicle ID based on mode_1 values
# 3 - 12 are various household vehicles
daysim_tour['Vehid'] = 1

# Trptrans
daysim_tour['Trptrans'] = daysim_tour['tmodetp'].map(mode_dict)

# Vehtype	Type of vehicle used (see 2001 NHTS codebook for VEHTYPE)
# FIXME: look this up from Elmer
daysim_tour['Vehtype'] = 1
# 01=Car
# 02=Van
# 03=SUV
# 04=Pickup truck
# 05=Other truck
# 06=RV
# 07=Motorcycle
# 91=Other

# Persons
# Fixme look up from Elmer
daysim_tour['Persons'] = 1

# HhVehUsed	Whether household vehicle used (1=yes, 2=no)
# FIXME: look this up from Elmer
daysim_tour['HhVehUsed'] = 1




# Disttowk	Distance from home to work for the person on the tour who works farthest from home
daysim_tour['PERSONID'] = daysim_tour['pno']
daysim_tour = daysim_tour.merge(daysim_person[['hhno','pno','pwaudist']], on=['hhno','pno'], how='left')
daysim_tour['Disttowk'] = daysim_tour['pwaudist'].copy()


In [703]:
daysim_trip = pd.read_csv(os.path.join(daysim_loc,'_trip.tsv'), delim_whitespace=True)

c:\Users\Modeller\.conda\envs\summary\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [627]:
# daysim_tour

In [704]:
# Whyto	String contenating successive activity codes at trip end (see 2001 NHTS codebook for WHYTO)

# Get concatenation from trips



daysim_trip['Whyto'] = daysim_trip['dpurp'].map(purp_dict).astype('str')
df = daysim_trip.groupby(['hhno','pno','day','tour']).agg({'Whyto': lambda x: '-'.join(set(x))}).reset_index()
daysim_tour = daysim_tour.merge(df, on=['hhno','pno','day','tour'], how='left')
# -7=Refused 64 19,794,097
# -8=Don't Know 259 132,632,045
# -9=Not Ascertained 126 44,702,675
# 01=Home 219,701 137,689,298,480
# 11=Go to work 47,975 29,451,746,910
# 12=Return to work 11,141 7,613,650,347
# 13=Attend business meeting/trip 1,363 871,580,135
# 14=Other work related 11,719 7,907,627,500
# 20=School/religious activity 4,019 2,042,102,857
# 21=Go to school as student 18,557 12,580,912,637
# 22=Go to religious activity 9,925 6,748,570,907
# 23=Go to library: school related 868 611,747,616
# 24=OS - Day care 2,487 1,946,703,725
# 30=Medical/dental services 9,241 5,591,347,518
# 40=Shopping/errands 18,445 9,664,073,513
# 41=Buy goods: groceries/clothing/hardware store 72,946 45,833,056,825
# 42=Buy services: video rentals/dry cleaner/post office/car service/bank
# 43=Buy gas 9,536 6,527,025,830
# 50=Social/recreational 7,042 3,361,065,902
# 51=Go to gym/exercise/play sports 19,898 12,826,376,945
# 52=Rest or relaxation/vacation 2,916 1,867,012,600
# 53=Visit friends/relatives 30,680 20,272,463,086
# 54=Go out/hang out: entertainment/theater/sports event/go to bar
# 55=Visit public place: historical site/museum/park/library 2,775 1,667,634,692
# 60=Family personal business/obligations 9,990 6,123,424,057
# 61=Use professional services: attorney/accountant 1,360 952,077,862
# 62=Attend funeral/wedding 1,133 733,967,929
# 63=Use personal services: grooming/haircut/nails 2,384 1,530,137,567
# 64=Pet care: walk the dog/vet visits 2,689 1,681,188,074
# 65=Attend meeting: PTA/home owners association/local government 3,139 1,866,480,387
# 70=Transport someone 801 466,247,452
# 71=Pick up someone 17,431 11,382,208,501
# 72=Take and wait 3,910 3,163,771,957
# 73=Drop someone off 19,085 12,587,299,015
# 80=Meals 3,999 1,383,289,688
# 81=Social event 3,201 2,168,662,259
# 82=Get/eat meal 30,313 21,394,703,354
# 83=Coffee/ice cream/snacks 3,844 2,611,957,719
# 91=Other reason

In [705]:
# daysim_tour['Whyto']

In [706]:
df = daysim_tour[nhts_tour.columns]
pyreadr.write_rdata(output_dir + '\\ToursByHh_df.Rda', df, df_name="ToursByHh_df")

# Trips

In [707]:
# Load NHTS survey files
fname = r'C:\Workspace\VisionEval_build\VisionEval\sources\modules\VE2001NHTS\data-raw\Dt_df.Rda'
nhts_trip = pyreadr.read_r(fname)['Dt_df']

In [708]:
nhts_trip.columns

Index(['HOUSEID', 'PERSONID', 'VEHID'], dtype='object')

In [709]:
daysim_trip.columns

Index(['hhno', 'pno', 'day', 'tour', 'half', 'tseg', 'tsvid', 'opurp', 'dpurp',
       'oadtyp', 'dadtyp', 'opcl', 'dpcl', 'otaz', 'dtaz', 'mode', 'pathtype',
       'dorp', 'deptm', 'arrtm', 'endacttm', 'trexpfac', 'id', 'travtime',
       'travcost', 'travdist', 'sov_ff_time', 'Whyto'],
      dtype='object')

In [710]:
# 'HOUSEID', 
daysim_trip['HOUSEID'] = daysim_trip['hhno'].astype('str')
# 'TDCASEID', 
daysim_trip['TDCASEID'] = range(1,len(daysim_trip)+1)

# 'VEHID', 
# FIXME: see if we can get this data from Elmer
daysim_trip['VEHID'] = 1

# 'VEHUSED', 
daysim_trip['VEHUSED'] = 2
daysim_trip.loc[daysim_trip['mode'].isin([3,4,5]), 'VEHUSED'] = 1

# 'TRPHHVEH', 
# assuming all vehicles used are hh vehicles
daysim_trip['TRPHHVEH'] = daysim_trip['VEHID'].copy()

# 'PERSONID',
daysim_trip['PERSONID'] = daysim_trip['hhno'].astype('str')+daysim_trip['pno'].astype('str')

# 'NUMONTRP', 
# FIXME: Get from Elmer
daysim_trip['NUMONTRP'] = 1

# 'TRPTRANS', 
daysim_trip['TRPTRANS'] = daysim_trip['mode'].map(mode_dict)

# 'TRPMILES', 
daysim_trip['TRPMILES'] = daysim_trip['travdist'].copy()

# 'TRVL_MIN', 
daysim_trip['TRVL_MIN'] = daysim_trip['travtime'].copy()

# 'DWELTIME', 
# FIXME: need to get this from Elmer
daysim_trip['DWELTIME'] = 10

# 'PSGR_FLG',
daysim_trip['PSGR_FLG'] = 2
daysim_trip.loc[(daysim_trip['dorp'] == 1) & (daysim_trip['mode'].isin([3,4,5])), 'PSGR_FLG'] = 1

# 'WHYFROM', 
daysim_trip['WHYFROM'] = daysim_trip['opurp'].map(purp_dict)

# 'WHYTO', 
daysim_trip['WHYTO'] = daysim_trip['dpurp'].map(purp_dict)

# 'VEHTYPE'
# FIXEME: get this info from Elmer
daysim_trip['VEHTYPE'] = 1

In [711]:
output_dir + '\\Dt_df.Rda'

'C:\\Workspace\\VisionEval\\psrc_scripts\\survey_data\\Dt_df.Rda'

In [713]:
df = daysim_trip[nhts_trip.columns]
pyreadr.write_rdata(output_dir + '\\Dt_df.Rda', df, df_name="Dt_df")

# Vehicles

# Load NHTS survey files


In [595]:
fname = r'C:\Workspace\VisionEval_build\VisionEval\sources\modules\VE2001NHTS\data-raw\Veh_df.Rda'
nhts_veh = pyreadr.read_r(fname)['Veh_df']

In [596]:
nhts_veh.columns

Index(['HOUSEID', 'VEHID', 'BESTMILE', 'EIADMPG', 'GSCOST', 'VEHTYPE',
       'VEHYEAR', 'VEHMILES'],
      dtype='object')

In [601]:
import pyodbc
import pandas as pd

conn_string = "DRIVER={ODBC Driver 17 for SQL Server}; SERVER=AWS-PROD-SQL\Sockeye; DATABASE=Elmer; trusted_connection=yes"
sql_conn = pyodbc.connect(conn_string)
df = pd.read_sql(sql='select * from HHSurvey.v_vehicles where survey_year=2017 or survey_year=2019', con=sql_conn)

In [602]:
df['HOUSEID']  = df['household_id'].astype('str')
df['VEHID'] = df['vehnum'].copy()

# BESTMILE
# Best estimate of annual miles
df['BESTMILE'] = 10000 # use average number, not sure how to get this info...

# FIXME: look up this infro from some EPA database on mileage?
# How did SANDAG do this?
df['EIADMPG'] = 20

# USing average here
df['GSCOST'] = 350

# FIXME: create a lookup for all the make and models
df['VEHTYPE'] = 1
# -8	I don't know	153	0.1
# 01	Automobile/Car/Station Wagon	122,391	47.8
# 02	Van (Mini/Cargo/Passenger)	13,184	5.1
# 03	SUV (Santa Fe, Tahoe, Jeep, etc.)	60,353	23.6
# 04	Pickup Truck	46,232	18.1
# 05	Other Truck	1,414	0.6
# 06	RV (Recreational Vehicle)	2,097	0.8
# 07	Motorcycle/Motorbike	9,283	3.6
# 97	Something Else
# df.groupby(['make','model']).count().index

df['VEHYEAR'] = df['year'].copy()

# FIXME: get an average number of this?
df['VEHMILES'] = 50000

In [603]:
df = df[nhts_veh.columns]
pyreadr.write_rdata(output_dir + '\\Veh_df.Rda', df, df_name="Veh_df")

# Clean Up
Make sure all household have some tours or trips (?)


# Load NHTS survey files


In [715]:
fname = output_dir + '\\Hh_df.Rda'
df_hh = pyreadr.read_r(fname)['Hh_df']

fname = output_dir + '\\ToursByHh_df.Rda'
df_tour = pyreadr.read_r(fname)['ToursByHh_df']

fname = output_dir + '\\Per_df.Rda'
df_person = pyreadr.read_r(fname)['Per_df']

df_hh = df_hh[df_hh['HOUSEID'].isin(df_tour['Houseid'])]
df_person = df_person[df_person['HOUSEID'].isin(df_hh['HOUSEID'])]

pyreadr.write_rdata(output_dir + '\\Hh_df.Rda', df_hh, df_name="Hh_df")
pyreadr.write_rdata(output_dir + '\\Per_df.Rda', df_hh, df_name="Per_df")

KeyError: 'ToursByHh_df'